In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import os

In [ ]:
step_length = 1    # The step length we take to get our samples from our corpus
epochs = 50       # Number of times we train on our full data
batch_size = 32    # Data samples in each training step
latent_dim = 64    # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./poke_gen_model.h5') # Location for the model
load_model = False # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1      # Print result for each epoch
gen_amount = 10    # How many 

In [ ]:
input_path = os.path.realpath('names.txt')

input_names = []

print('Reading names from file:')
with open(input_path) as f:
    for name in f:
        name = name.rstrip()
        if len(input_names) < 10:
            print(name)
        input_names.append(name)
    print('...')

Reading names from file:
Abbas
Abbey
Abbott
Abdi
Abel
Abraham
Abrahams
Abrams
Ackary
Ackroyd
...


In [ ]:
# Make it all to a long string
concat_names = '\n'.join(input_names).lower()

# Find all unique characters by using set()
chars = sorted(list(set(concat_names)))
num_chars = len(chars)

# Build translation dictionaries, 'a' -> 0, 0 -> 'a'
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

# Use longest name length as our sequence window
max_sequence_length = max([len(name) for name in input_names])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(input_names))
print('Longest name: ', max_sequence_length)

Total chars: 28
Corpus length: 27124
Number of names:  3668
Longest name:  14


In [ ]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}]   y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 27110
First 10 sequences and next chars:
X=[abbas abbey ab]   y=[b]
X=[bbas abbey abb]   y=[o]
X=[bas abbey abbo]   y=[t]
X=[as abbey abbot]   y=[t]
X=[s abbey abbott]   y=[ ]
X=[ abbey abbott ]   y=[a]
X=[abbey abbott a]   y=[b]
X=[bbey abbott ab]   y=[d]
X=[bey abbott abd]   y=[i]
X=[ey abbott abdi]   y=[ ]


In [ ]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
Y = np.zeros((num_sequences, num_chars), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char2idx[char]] = 1
    Y[i, char2idx[next_chars[i]]] = 1
    
print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))

X shape: (27110, 14, 28)
Y shape: (27110, 28)


<ipython-input-18-4e755f62cb62>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
<ipython-input-18-4e755f62cb62>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = np.zeros((num_sequences, num_chars), dtype=np.bool)


In [ ]:
model = Sequential()
model.add(LSTM(latent_dim, 
               input_shape=(max_sequence_length, num_chars),  
               recurrent_dropout=dropout_rate))
model.add(Dense(units=num_chars, activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 64)                23808     
                                                                 
 dense_1 (Dense)             (None, 28)                1820      
                                                                 
Total params: 25,628
Trainable params: 25,628
Non-trainable params: 0
_________________________________________________________________


In [ ]:
if load_model:
    model.load_weights(model_path)
else:
    
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
    
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)

Start training for 100 epochs
Epoch 1/100
848/848 [==============================] - 8s 8ms/step - loss: 2.2467
Epoch 2/100
848/848 [==============================] - 7s 8ms/step - loss: 1.7138
Epoch 3/100
848/848 [==============================] - 7s 8ms/step - loss: 1.4893
Epoch 4/100
848/848 [==============================] - 9s 10ms/step - loss: 1.3883
Epoch 5/100
848/848 [==============================] - 9s 10ms/step - loss: 1.3306
Epoch 6/100
848/848 [==============================] - 9s 11ms/step - loss: 1.2956
Epoch 7/100
848/848 [==============================] - 9s 10ms/step - loss: 1.2681
Epoch 8/100
848/848 [==============================] - 9s 10ms/step - loss: 1.2485
Epoch 9/100
848/848 [==============================] - 9s 11ms/step - loss: 1.2288
Epoch 10/100
848/848 [==============================] - 11s 13ms/step - loss: 1.2131
Epoch 11/100
848/848 [==============================] - 12s 14ms/step - loss: 1.1971
Epoch 12/100
848/848 [==============================] - 

848/848 [==============================] - 10s 11ms/step - loss: 1.0053
Epoch 99/100
848/848 [==============================] - 10s 11ms/step - loss: 1.0035
Epoch 100/100
848/848 [==============================] - 10s 12ms/step - loss: 1.0030
Finished training - time elapsed: 15.349590941270192 min
Storing model at: /home/haris/Documents/LSTM/poke_gen_model.h5


In [ ]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []

print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char2idx[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)   
    next_char = idx2char[next_idx]   
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':

        gen_name = [name for name in sequence.split('\n')][1]

        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]

        # Discard all names that are too short
        if len(gen_name) > 2:
            
            # Only allow new and unique names
            if gen_name not in input_names + new_names:
                new_names.append(gen_name.capitalize())

        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

10 new names are being generated
Generated 1
Generated 2
Generated 3
Generated 4
Generated 5
Generated 6
Generated 7
Generated 8
Generated 9
Generated 10


In [ ]:
print_first_n = min(10, gen_amount)

print('First {} generated names:'.format(print_first_n))
for name in new_names[:print_first_n]:
    print(name)


First 10 generated names:
Zaoui
Palner
Palner
Pane
Panrett
Panm
Parner
Parrey
Parrett
Parrison
